Osnabrück University - Machine Learning (Summer Term 2024) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack, Lukas Niehaus

# Exercise Sheet 07

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, June 2th, 2023**. If you need help (and Google and other resources were not enough), ask in the forum, contact your groups designated tutor or whomever of us you run into first. Please upload your results to your group's studip folder.

## Assignment 0: Math recap (Hyperplanes) [0 Points]

This exercise is supposed to be very easy and is voluntary. There will be a similar exercise on every sheet. It is intended to revise some basic mathematical notions that are assumed throughout this class and to allow you to check if you are comfortable with them. Usually you should have no problem to answer these questions offhand, but if you feel unsure, this is a good time to look them up again. You are always welcome to discuss questions with the tutors or in the practice session. Also, if you have a (math) topic you would like to recap, please let us know.

**a)** What is a *hyperplane*? What are the hyperlanes in $\mathbb{R}^2$ and $\mathbb{R}^3$? How are the usually described?

Hyperplanes are **subspaces** which have **one dimension less** than the space they are in. That is, hyperplanes in $\mathbb{R}^3$ are 2-dimensional planes, while in $\mathbb{R}^2$ they are 1-dimensional lines. 
In this imaginable dimensions you can see that hyperplanes will split the space in two parts.

If you are in $\mathbb{R}^n$ a hyperplane is described by an equation
$$ a_1 x_1 + a_2 x_2 + ... + a_n x_n = b$$
where at least one $a_i$ is non-zero and $b$ is a constant. (Remark: the description of a line as a function in the form $y=mx+b$, as you may know it from school, is recovered as a special case, when you set $x_1=x$, $x_2=y$, $a_1=-m$, and $a_2=1$).

**b)** What is the Hesse normal form? What is the intuition behind? What are its advantages?

The **Hesse normal form** is a way to define a hyperplane. It is given by the following equation

$$\vec{x} \cdot \vec{n_0} - d = 0$$

$\vec{n_0}$ is the unit normal vector that points from the **origin** to the plane and is orthogonal to the plane. $d$ describes the distance from the origin to the plane.

The hesse normal form is basically the same as the equation from exercise a). The only difference is that it's normalized ($n_0$ has length 1 and thus $d$ is the distance). This helps, for example, at computing the distance from a point to a hyperplane.

**c)** Can you transform the standard form of a hyperplane into the Hesse normal form and vice versa?

If you got the standard form
$$ a_1 x_1 + a_2 x_2 + ... + a_n x_n = b$$
you can compute 
$$ \vec{n_0} = \frac{\vec{a}}{|\vec{a}|} \\ d = \frac{b}{|\vec{a}|} $$
The standard form is not recoverable from the normal form as there is not one standard form, but infinite many which would all be *normed* to the same Hesse normal form.

## Assignment 1: Local PCA (7 points)

In the lecture we learned that regular PCA is ill suited for special cases of data. In this assignment we will take a look at local PCA which is used for clustered data (ML-06, Slide 25). This is mostly a repetition of algorithms we already used. Feel free to use the built-in functions for k-means clustering and PCA from the libraries (we already included the right imports to set you on track). See:
* For K-Means
    * [scipy.cluster.vq.vq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.vq.html)
    * [scipy.cluster.vq.kmeans](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.kmeans.html)
* For PCA    
    * [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
* To plot arrows
    * [matplotlib.pyplot.quiver](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html)
* Numpy
    * [@ / np.matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html#numpy.matmul)
    * [.T / np.transpose](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html#numpy.transpose)

In [ ]:
%matplotlib ipympl

import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt

from numpy.random import multivariate_normal as multNorm

from scipy.cluster.vq import kmeans, vq
from sklearn.decomposition import PCA

np.random.seed(42)

# Generate clustered data - you may plot the data to take a look at it
data = np.vstack((multNorm([2, 2], [[0.1, 0], [0, 1]], 100),
                  multNorm([-2, -4], [[1, 0], [0, 0.3]], 100)))

# TODO: Apply k-means to the data.
### BEGIN SOLUTION
# Compute k-means with K = 2 (2 clusters).
centroids, _ = kmeans(data, 2)

# Assign each sample to a cluster.
idx, _ = vq(data, centroids)
### END SOLUTION

# TODO: Apply PCA for each cluster and store each two largest components.
### BEGIN SOLUTION
pcs = np.empty([np.unique(idx).size, 2, 2])
for idxU in np.unique(idx):
    # Create PCA instance.
    pca = PCA(2)
    # Fit the data to the PCA instance
    pca.fit(data[idx == idxU])
    # Retrieve, scale and store the components
    pcs[idxU] = (pca.components_.T * np.sqrt(pca.explained_variance_)).T
### END SOLUTION

# TODO: Plot the results of k-means and local PCA
### BEGIN SOLUTION
fig, ax = plt.subplots(num='Local PCA')

# make cluster centroids the same format as the PCs
cls = np.tile(centroids[np.newaxis], (pcs.shape[0], 1, 1))

# plot data, PCs and centroids
for i, idxU in enumerate(np.unique(idx)):
    ax.plot(*zip(*data[idx == idxU]), 'o', zorder=-1, alpha=0.2)
    # scale length of vecvtors for better visualization
    ax.quiver(*cls[:, i, :].T, *pcs[i, :, :].T, zorder=1, angles='xy', scale_units='xy', scale=1)

ax.plot(centroids[:, 0], centroids[:, 1], '*y', markersize=12, alpha=1, zorder=.5)

ax.axis('equal')
plt.show()
### END SOLUTION

## Assignment 2: Projection Pursuit (3 points)

**a)** Explain in your own words the idea of projection pursuit. Is it a linear or non-linear method for dimension reduction? Discuss why high variance, non-Gaussianness and clusters indicate an interesting feature. What is the relation to PCA? 

Projection pursuit aims at finding good projection axes. It is typically applied for linear dimension reduction (though in principle the idea could also be applied to non-linear projections). What exactly a good projection axis is, obviously depends on the specific task, but there are some general patterns which can be considered for projection pursuit:
* assuming a decent signal to noise ratio, the actual signal can be expected to have higher variance than the noise and hence high variance projections are more likely to capture the actual signal instead of noise
* unimodal distributions, and especially Gaussian distributed data are not well suited when one is aiming at decision making, that is, splitting the data into different parts, e.g. for clustering (unsupervised) or classification (supervised).
* more generally, if projected data already exhibits a clusters chances are high that these reflect interesting structure of the data.

In contrast to PCA, which simply aims at maximizing the variance, projection pursuit also takes these other aspects into account.

**b)** Explain how the different indices (Friedman-Tukey, Hermite, Natural Hermite, and Entropy) detect interesting features and discuss advantages and disadvantages.

All projection indices aim at identifying "interesting" projections of the data, by assigning low values to "uninteresting" directions and high values to "interesting" directions.  The indices differ in what they consider to be "interesting":

* The projection index proposed by *Friedman and Tukey* (1974) is formed as a product $I(\vec{k})=s(\vec{k})\cdot p(\vec{k})$ with $s(\vec{k})$ quantifying the variance of the projected data and $p(\vec{k})$ being the average distance between points along $\vec{k}$. Large values of $s(\vec{k})$ ensure that the direction covers the dominant direction of the data, while a large $p(\vec{k})$ value indicates that data form clusters along that axis.
* Non-Gaussianness makes a direction "interesting", as it allows for separating data along this axis. Hence the *Hermite index* explicitly quantifies the (squared) deviation from the normal distribution, making a direction the more interesting the less the projected data is normally distributed.
* The *natural Hermite index* follows the same idea, putting more emphasis on the central parts of the distribution.
* Huber (1985) argues that for independent variables $X$ and $Y$, a good index should satisfy $I(X+Y)\leq \max(I(X),I(Y))$ and shows that a large class of indices can be constructed as $I(X)=h(S_1(X)/S_2(X))$ with monotone increasing $h$, $S_1$ subadditive and $S_2$ superadditive. He then proposes the (negative) *entropy index* as one example of such an index.

While all of these indices tend to detect similar non-normal directions, they react differently to ouliers. A good strategy, already proposed by Friedman and Tukey (1974), is to use a trimmed version of the index, that is obtained by removing the most extreme points from the projected data before computing the index value.

**c)** Explain the idea of projection pursuit for clustering (ML-06, slides 56-58). How is the index computed and why does maximizing that index yield good clusters?

Standard approaches to clustering use all variables of the data, as the similarity metrics (e.g., Euclidean) are definined in the full dimension of the data space.  This may lead to suboptimal clusters, for examples if the cluster structure is contained in a subspace of the data while other dimensions contain noise hiding the actual cluster structure.  Projection pursuit clustering aims at recovering clusters in lower dimensional subspaces by simultanously performing dimension reduction and clustering.

Projection Pursuit Clustering aims at finding a direction $\vec{k}$ that maximizes the projection index (e.g. Friedman-Tukey, see above) and simultaniously forms clusters. Clustering proceeds in a non-hierarchical fashion, assuming a fixed number of $K$ clusters, to which (projected) data are assigned such that the within-group sum-of-square distance is minimized.

## Assignment 3: Hebbian Learning (6 points)

In the lecture (ML-07, Slides 10ff.) there is a simplified version of Ivan Pavlov's famous experiment on classical conditioning. In this exercise you will take a look into this simplified model and create your own conditionable dog with a simple Hebbian learning rule.

### a) Programming a Dog
To model the dog salivation behavior we will need to model an unconditioned and a conditioned stimulus: food and bell. They are represented as lists: `weight_food` and `weight_bell`. Note that one could just use a single number, the lists are only here to keep track of the history for a nice output. It is possible to access the current weight by selecting the last item of each list, respectively: `weight_food[-1]`.

A list of trials is already given as well as a condition database. Each entry represents an index to select from the `condition_db`. To figure out the value of the stimulus `food` in the second trial (which maps to condition `1`) one could do: `condition_db[1]["food"]`.

Your task is to implement a `for` loop over all trials. In each iteration select the correct values for $x_1$ and $x_2$ from the condition database and retrieve the current weights $w_1$ and $w_2$. Then calculate the response of the dog with the threshold $\theta$:

$$
r_t = \Theta(x_{1,t} w_{1,t-1} + x_{2,t} w_{2,t-1})\\
\Theta(x)= \begin{cases}1 \text{ if } x >= \theta\\0 \text{ else }\end{cases}
$$

With this response calculate both $w_{n,t}$ according to the Hebbian rule:

$$w_{n,t} = w_{n, t-1} + \epsilon \cdot r_t \cdot x_{n,t}$$

*Note: While you program the output might look a little messy, don't worry about it. Once you fill up all three lists properly, it will look much like on ML-07, Slide 14.*

*Hint: The [list.append()](https://docs.python.org/3/tutorial/datastructures.html) function is probably rather useful.*

In [ ]:
# Initialization
condition_db = [{"food": 1, "bell": 0}, 
                {"food": 0, "bell": 1},
                {"food": 1, "bell": 1}]

trials = [0, 1, 2, 1, 2, 1, 2, 1]

epsilon = 0.2
theta = 1/2

responses = []
weight_food = [1]
weight_bell = [0]

# TODO: For each trial, update the current weights of the US and CS and store
# the results in the respective lists. Also store the response.
### BEGIN SOLUTION
for trial in trials:
    # Retrieve the data
    x_1 = condition_db[trial]["food"]
    x_2 = condition_db[trial]["bell"]
    w_1 = weight_food[-1]
    w_2 = weight_bell[-1]
    
    # Python interprets True as 1 and False as 0, so we can actually
    # just store the evaluation of r instead of doing an if/else
    response = x_1 * w_1 + x_2 * w_2 >= theta
    responses.append(response)
    
    weight_food.append(w_1 + epsilon * response * x_1)
    weight_bell.append(w_2 + epsilon * response * x_2)
### END SOLUTION

# Output
print("| Food   |   |" + "|   |".join(["{:3d}".format(condition_db[trial]["food"]) for trial in trials]) + "|   |")
print("| Bell   |   |" + "|   |".join(["{:3d}".format(condition_db[trial]["bell"]) for trial in trials]) + "|   |")
print("| Saliva |   |" + "|   |".join(["{:3d}".format(response) for response in responses]) + "|   |")
print("| w_Food |" + "|   |".join(["{:3.1f}".format(w) for w in weight_food]) + "|")
print("| w_Bell |" + "|   |".join(["{:3.1f}".format(w) for w in weight_bell]) + "|")

### b) Parameter adjustment

In the above default setting of trials (`[0, 1, 2, 1, 2, 1, 2, 1]`, in case you changed it), how many learning steps, i.e. simultaneous presentation of unconditioned and conditioned stimulus, did you need until the dog started to produce saliva on the conditioned stimulus? What happens if you change the parameters $\epsilon$ and $\theta$? Try smaller and bigger values for each or present different conditions to the dog.

Three learning steps were needed to train the dog to react to the conditioned stimulus.

With a smaller (but positive) epsilon the number of learning steps increases, while with a smaller threshold the number of learning steps decreases.
With a higher epsilon the number of learning steps decreases, while with a higher threshold (theta < 1, otherwise the dog won't even produce salvia when it's presented with food) the number of learning steps increases.

## Assignment 4: The Logic Perceptron (4 points)

### a) The Logic Perceptron

For the following two logical functions sketch the weights of a perceptron after it was trained. To do so, figure out when the perceptron should fire. Then come up with ideas of how you can achieve this. Provide the values for $w_0,w_1,w_2$ as well as some explanation.
the activation function evaluates to $< 0$ as negative and in the other cases $\ge 0$ as positive. Remember that $w_0$, the bias, can be adjusted to create the correct positive/negative classification in combination with the constant $x_0 = 1$.

#### 1) $(A \wedge B) \vee (\neg A \wedge B)$

The given function evaluates as follows:

  | $$A$$ | $$B$$ | $$A \wedge B$$ | $$\neg A \wedge B$$ | $$(A \wedge B) \vee (\neg A \wedge B)$$ |
  |-----|-----|--------------|-------------------|---------------------------------------|
  |  0  |  0  |      0       |         0         |                 0                     |
  |  0  |  1  |      0       |         1         |                 1                     |
  |  1  |  0  |      0       |         0         |                 0                     |
  |  1  |  1  |      1       |         0         |                 1                     |

Thus the function evaluates to true only if $B$ is true.

The perceptron activates if:

$$w_0 x_0 + w_1 x_1 + w_2 x_2 \ge 0$$

If $x_1$ is $A$ we can ignore it for example by setting $w_1 = 0$. To count $B$ we can adjust the weight $w_2 = 1$. So now we get the following activation function evaluations:

$$\begin{align*}
x = (0, 0) &: \quad 1 w_0 + 0 w_1 + 0 w_2 = 0 + w_0 \\
x = (0, 1) &: \quad 1 w_0 + 0 w_1 + 1 w_2 = 1 + w_0 \\
x = (1, 0) &: \quad 1 w_0 + 1 w_1 + 0 w_2 = 0 + w_0 \\
x = (1, 1) &: \quad 1 w_0 + 1 w_1 + 1 w_2 = 1 + w_0
\end{align*}$$

To fire on $B$ we can set the bias to $w_0 = -0.5$ such that for $x = (0, 0)$ and $x = (1, 0)$ the activation functions evaluate to values $< 0$ and in the other cases $\ge 0$.

#### 2) $(A \wedge B) \vee (\neg A \wedge B) \vee (A \wedge \neg B)$

The given function evaluates as follows:

| $$A$$ | $$B$$ | $$A \wedge B$$ | $$\neg A \wedge B$$ | $$  A \wedge \neg B$$ | $$ \begin{align*}&(A \wedge B)\\ \vee &(\neg A \wedge B)\\ \vee &(A \wedge \neg B) \end{align*} $$ |
|-----|-----|--------------|-------------------|-------------------|-----------|
|  0  |  0  |      0       |         0         |         0         |     0     |
|  0  |  1  |      0       |         1         |         0         |     1     |
|  1  |  0  |      0       |         0         |         1         |     1     |
|  1  |  1  |      1       |         0         |         0         |     1     |

Thus the function evaluates to true if any of the variables $A$ or $B$ is true, but false if both are false.

The perceptron activates if:

$$w_0 x_0 + w_1 x_1 + w_2 x_2 \ge 0$$

If $x_1$ is $A$ we can set $w_1 = 1$. To count $B$ we can adjust the weight $w_2 = 1$. So now we get the following activation function evaluations:

$$\begin{align*}
x = (0, 0) &: \quad 1 w_0 + 0 w_1 + 0 w_2 = 0 + w_0 \\
x = (0, 1) &: \quad 1 w_0 + 0 w_1 + 1 w_2 = 1 + w_0 \\
x = (1, 0) &: \quad 1 w_0 + 1 w_1 + 0 w_2 = 1 + w_0 \\
x = (1, 1) &: \quad 1 w_0 + 1 w_1 + 1 w_2 = 2 + w_0
\end{align*}$$

To fire on $B$ we can set the bias to $w_0 = -0.5$ such that for $x = (0, 0)$ the activation function evaluates to $< 0$ and in the other cases $\ge 0$.